Step 2: Data Cleaning & Validation

Create Spark Session

In [16]:
# code for Data cleaning and validation
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, when, count, isnan

spark = SparkSession.builder \
    .appName("Crypto Data Cleaning") \
    .getOrCreate()


Load CSV File

In [17]:
url = r"C:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\PROJECTS\Academic_project\BDA_BITCOIN\CryptoPredict\data\synthetic_bitcoin_2014_2023.csv"
df = spark.read.csv(url, header=True, inferSchema=True)
df.show(5)
print(f"Total Rows: {df.count()}")


+----------+------------------+------------------+------------------+------------------+------+------------------+
|      Date|              Open|              High|               Low|             Close|Volume|         MarketCap|
+----------+------------------+------------------+------------------+------------------+------+------------------+
|2014-01-01|20968.082996411034| 20829.91171069015|20870.791135428568|20945.270521984607| 19033|3986513.3384493305|
|2014-01-02| 19522.47194008754|19797.146770887925|19423.537425631155|19674.704767049698| 36088| 7100207.456332895|
|2014-01-03| 20701.45138135251| 20785.33115376397|20329.565015906777| 20650.36962066113| 44465| 9182186.851826971|
|2014-01-04|  20938.4878660132| 20980.01940620715| 20754.12859635343|20866.669464301012| 18527|3865967.8516510488|
|2014-01-05|20456.964154911486|20312.142864650312|20309.665064519202| 20509.47348177957| 12964|2658848.1421779036|
+----------+------------------+------------------+------------------+-----------

Handle Missing / Null Values

In [18]:
from pyspark.sql.functions import col, when, count

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()


+----+----+----+---+-----+------+---------+
|Date|Open|High|Low|Close|Volume|MarketCap|
+----+----+----+---+-----+------+---------+
|   0|   0|   0|  0|    0|     0|        0|
+----+----+----+---+-----+------+---------+



Remove Duplicates

In [19]:
before = df.count()
df_no_duplicates = df.dropDuplicates()
after = df_no_duplicates.count()
print(f"Removed {before - after} duplicate rows")


Removed 0 duplicate rows


Detect & Remove Outliers (on “Close”)

In [20]:
stats = df_no_duplicates.select(
    mean(col("Close")).alias("mean"),
    stddev(col("Close")).alias("stddev")
).collect()

mean_val = stats[0]["mean"]
std_val = stats[0]["stddev"]

print(f"Mean Close Price: {mean_val:.2f}, Std Dev: {std_val:.2f}")

df_no_outliers = df_no_duplicates.filter(
    (col("Close") >= mean_val - 3 * std_val) &
    (col("Close") <= mean_val + 3 * std_val)
)

print(f"After removing outliers: {df_no_outliers.count()} rows")


Mean Close Price: 20518.57, Std Dev: 1012.62
After removing outliers: 3642 rows


Validate Cleaned Data

In [21]:
df_no_outliers.describe(["Open", "High", "Low", "Close", "Volume"]).show()

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|              Open|              High|               Low|             Close|            Volume|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|              3642|              3642|              3642|              3642|              3642|
|   mean| 20514.83483401755|20616.696558847096| 20412.37077296476| 20514.97795462582|24989.030203185062|
| stddev|1000.3915896959207|1004.2657343796701|1004.2033573771114| 1000.015953448771|14303.377773427568|
|    min|17554.468854404855| 17552.02489278787|17327.930855040104|17483.292078958923|              1010|
|    max| 23528.89543327984|23808.953125325203|23406.057555410658| 23519.65383486466|             49975|
+-------+------------------+------------------+------------------+------------------+------------------+



Save Cleaned Data (optional)

In [22]:
df_no_outliers.toPandas().to_csv(r"C:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\PROJECTS\Academic_project\BDA_BITCOIN\CryptoPredict\data\cleaned_bitcoin_data.csv", index=False)


Step 3: Feature Engineering

In [3]:
# Feature Engineering for feature extraction 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, date_format, lag
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer

# --- 1. Setup and Data Loading ---

# Initialize Spark Session
spark = SparkSession.builder.appName("BitcoinAdvancedFeatures").getOrCreate()

# Load the dataset
file_path = r"C:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\PROJECTS\Academic_project\BDA_BITCOIN\CryptoPredict\data\cleaned_bitcoin_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Preprocessing: Convert 'Date' and cast columns to double
df = df.withColumn("Date", col("Date").cast("date"))
cols_to_cast = ["Open", "High", "Low", "Close", "Volume", "MarketCap"]
for c in cols_to_cast:
    df = df.withColumn(c, col(c).cast("double"))

# Order by Date for all time series operations
df = df.orderBy("Date")

In [4]:
# --- 2. Advanced Feature Engineering ---

# Define a general Window spec ordered by Date
window_spec_lag = Window.orderBy("Date")

# A. Lag Features (Prices and Volume)
lag_features_base = ["Close", "Open", "High", "Low", "Volume"]
lag_periods = [1, 2, 3, 7]

for feature in lag_features_base:
    for i in lag_periods:
        new_col_name = f"{feature}Lag{i}"
        df = df.withColumn(
            new_col_name,
            lag(col(feature), i).over(window_spec_lag)
        )

# B. Moving Average (7-day Simple Moving Average on Close)
# Define window for 7-day trailing average (6 preceding rows + current row)
window_spec_ma = Window.orderBy("Date").rowsBetween(-6, 0)
df = df.withColumn(
    "Close_SMA_7",
    F.avg(F.col("Close")).over(window_spec_ma)
)

# C. Log Returns (Log(Close_t / Close_t-1))
# Need the Close price from the previous day for the calculation
df = df.withColumn(
    "Prev_Close",
    F.lag(F.col("Close"), 1).over(window_spec_lag)
)
df = df.withColumn(
    "Log_Returns",
    F.log(F.col("Close") / F.col("Prev_Close"))
).drop("Prev_Close") # Drop the temporary lag column

# D. Volume Ratio (Volume / MarketCap)
df = df.withColumn(
    "Volume_to_MarketCap_Ratio",
    F.col("Volume") / F.col("MarketCap")
)

# E. Time-Based Features
df = df.withColumn("DayOfWeek", date_format(col("Date"), "E"))
df = df.withColumn("Month", date_format(col("Date"), "M").cast("int"))


In [5]:
# --- 3. Prepare X and Y DataFrames ---

# 1. Drop rows with any Nulls (due to lag, SMA, and log returns calculations)
df_clean = df.na.drop()

# 2. StringIndex the categorical 'DayOfWeek' column
indexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeek_Index")
df_indexed = indexer.fit(df_clean).transform(df_clean)

# 3. Define the Target DataFrame (Y)
target_col = "Close"
y_df = df_indexed.select("Date", col(target_col).alias("Target"))

# 4. Define the Feature DataFrame (X)
# X includes all engineered features: lag, MA, returns, ratio, and time features
engineered_features = [c for c in df_indexed.columns if 'Lag' in c or 'SMA' in c or 'Returns' in c or 'Ratio' in c]
x_cols_final = ["Date"] + engineered_features + ["DayOfWeek_Index", "Month"]
x_df = df_indexed.select(x_cols_final)

Step 4: Model Application 

In [6]:
# Convert x_df and y_df to Pandas (if they are Spark DataFrames)
x = x_df.toPandas()
y = y_df.toPandas()

print(x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3635 entries, 0 to 3634
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       3635 non-null   object 
 1   CloseLag1                  3635 non-null   float64
 2   CloseLag2                  3635 non-null   float64
 3   CloseLag3                  3635 non-null   float64
 4   CloseLag7                  3635 non-null   float64
 5   OpenLag1                   3635 non-null   float64
 6   OpenLag2                   3635 non-null   float64
 7   OpenLag3                   3635 non-null   float64
 8   OpenLag7                   3635 non-null   float64
 9   HighLag1                   3635 non-null   float64
 10  HighLag2                   3635 non-null   float64
 11  HighLag3                   3635 non-null   float64
 12  HighLag7                   3635 non-null   float64
 13  LowLag1                    3635 non-null   float

In [7]:
# Drop Date column (not needed for modelling)
X = x.drop(columns=["Date"])

# Extract Target column
y = y["Target"]


In [20]:
# Prepare for raw data or unfeatured data and uncleaned 
import pandas as pd 
df=pd.read_csv(r'C:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\PROJECTS\Academic_project\BDA_BITCOIN\CryptoPredict\data\synthetic_bitcoin_2014_2023.csv')
print(df.head())
x_raw = df[["Open", "High", "Low", "Volume", "MarketCap"]]
y_raw = df["Close"]


         Date          Open          High           Low         Close  Volume  \
0  2014-01-01  20968.082996  20829.911711  20870.791135  20945.270522   19033   
1  2014-01-02  19522.471940  19797.146771  19423.537426  19674.704767   36088   
2  2014-01-03  20701.451381  20785.331154  20329.565016  20650.369621   44465   
3  2014-01-04  20938.487866  20980.019406  20754.128596  20866.669464   18527   
4  2014-01-05  20456.964155  20312.142865  20309.665065  20509.473482   12964   

      MarketCap  
0  3.986513e+06  
1  7.100207e+06  
2  9.182187e+06  
3  3.865968e+06  
4  2.658848e+06  


In [23]:
# Split training data and testing data 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Featured and cleared data 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Split raw Data
x_train_raw,x_test_raw,y_train_raw,y_test_raw=train_test_split(x_raw,y_raw,test_size=0.2,random_state=42)


Linear Regression

In [10]:
# Train Linear Regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Predict on Test set
y_pred_lr = lin_reg.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred_lr)
mse = mean_squared_error(y_test, y_pred_lr)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_lr)

print("📌 Linear Regression Performance:")
print(f"MAE  : {mae:.4f}")
print(f"MSE  : {mse:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"R²   : {r2:.4f}")


📌 Linear Regression Performance:
MAE  : 29.2604
MSE  : 1967.4146
RMSE : 44.3555
R²   : 0.9979


In [24]:
# Train Linear Regression model on RAW DATA
lin_reg_raw = LinearRegression()
lin_reg_raw.fit(x_train_raw, y_train_raw)

# Predict using RAW TEST DATA
y_pred_raw = lin_reg_raw.predict(x_test_raw)

# Evaluate performance (RAW)
mae_raw = mean_absolute_error(y_test_raw, y_pred_raw)
mse_raw = mean_squared_error(y_test_raw, y_pred_raw)
rmse_raw = np.sqrt(mse_raw)
r2_raw = r2_score(y_test_raw, y_pred_raw)

print("\n📌 PERFORMANCE (RAW DATA)")
print(f"MAE  : {mae_raw:.4f}")
print(f"MSE  : {mse_raw:.4f}")
print(f"RMSE : {rmse_raw:.4f}")
print(f"R²   : {r2_raw:.4f}")



📌 PERFORMANCE (RAW DATA)
MAE  : 31.1866
MSE  : 1525.1707
RMSE : 39.0534
R²   : 0.9986


RandomForestRegressor

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

#Define the Hyper parameter grid for RandomForestRegressor
param_grid={
    'n_estimators':[50,100,200],
    'max_depth':[None,10,20],
    'max_features':['sqrt','log2',None]
}

grid_search=GridSearchCV(
    estimator=RandomForestRegressor(random_state=32),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train,y_train)

#display best parameters ans score
print(f"Best Parameters from featured Data:{grid_search.best_params_}")

print(f"Best performance with Raw Data :{grid_search.fit(x_test_raw,y_test_raw).best_params_}")

c:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\.venv\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best Parameters from featured Data:{'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 50}
Best performance with Raw Data :{'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 50}


c:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\.venv\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [12]:
# Train RandomForestRegressor model 

rf_model=RandomForestRegressor(
    n_estimators=50,     # Number of trees
    max_depth=None,  
    max_features='sqrt',
    random_state=32)


rf_model.fit(X_train,y_train)

# Predict on Test set
y_pred_rf=rf_model.predict(X_test)

# Evaluate model performance
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse)
r2_rf = r2_score(y_test, y_pred_rf)

print("📌 RandomForestRegression Performance:")
print(f"MAE  : {mae_rf:.4f}")
print(f"MSE  : {mse_rf:.4f}")
print(f"RMSE : {rmse_rf:.4f}")
print(f"R²   : {r2_rf:.4f}")


📌 RandomForestRegression Performance:
MAE  : 79.6689
MSE  : 16268.7321
RMSE : 44.3555
R²   : 0.9824


In [ ]:
# Train RandomForestRegressor model 

rf_model_raw = RandomForestRegressor(
    n_estimators=50,     # Number of trees
    max_depth=None,  
    max_features='sqrt',
    random_state=32)

rf_model_raw.fit(x_train_raw, y_train_raw)

# Predict on Test set
y_pred_rf_raw = rf_model_raw.predict(x_test_raw)

# Evaluate model performance
mae_rf_raw = mean_absolute_error(y_test_raw, y_pred_rf_raw)
mse_rf_raw = mean_squared_error(y_test_raw, y_pred_rf_raw)
rmse_rf_raw = np.sqrt(mse_rf_raw)
r2_rf_raw = r2_score(y_test_raw, y_pred_rf_raw)

print("📌 RandomForestRegression Performance (RAW DATA):")
print(f"MAE  : {mae_rf_raw:.4f}")
print(f"MSE  : {mse_rf_raw:.4f}")
print(f"RMSE : {rmse_rf_raw:.4f}")
print(f"R²   : {r2_rf_raw:.4f}")


📌 RandomForestRegression Performance (RAW):
MAE  : 34.9328
MSE  : 1895.4806
RMSE : 43.5371
R²   : 0.9983


XGBoostRegressor

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

#  Base Model
model = XGBRegressor(random_state=42)

# Hyperparameter Grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

#  GridSearchCV
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,              # 3-fold Cross Validation
    scoring='r2',      # Can use 'mean_squared_error' also
    n_jobs=-1,         # Use all CPU cores
    verbose=1
)

grid.fit(X_train, y_train)     # Train grid search

print("\n🔥 Best Parameters:")
print(grid.best_params_)

# Evaluate Model
best_model = grid.best_estimator_   # Best XGBRegressor

y_pred_xgb = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse = np.sqrt(mse_xgb)
r2 = r2_score(y_test, y_pred_xgb)

print("\n📊 Evaluation Metrics for XGBRegressor :")
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R² Score:", r2)


Fitting 3 folds for each of 324 candidates, totalling 972 fits

🔥 Best Parameters:
{'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}

📊 Evaluation Metrics for XGBRegressor :
MAE: 6.34432484149086
MSE: 1967.414607217003
RMSE: 10.877388136832286
R² Score: 0.999872167613392


In [30]:
#  Base Model
model_raw = XGBRegressor(random_state=42)

#  GridSearchCV
grid_raw = GridSearchCV(
    estimator=model_raw,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_raw.fit(x_train_raw, y_train_raw)

print("\n🔥 Best Parameters (RAW):")
print(grid_raw.best_params_)

# Evaluate Model
best_model_raw = grid_raw.best_estimator_

y_pred_xgb_raw = best_model_raw.predict(x_test_raw)

mae_xgb_raw = mean_absolute_error(y_test_raw, y_pred_xgb_raw)
mse_xgb_raw = mean_squared_error(y_test_raw, y_pred_xgb_raw)
rmse_xgb_raw = np.sqrt(mse_xgb_raw)
r2_xgb_raw = r2_score(y_test_raw, y_pred_xgb_raw)

print("\n📊 Evaluation Metrics for XGBRegressor (RAW):")
print("MAE:", mae_xgb_raw)
print("MSE:", mse_xgb_raw)
print("RMSE:", rmse_xgb_raw)
print("R² Score:", r2_xgb_raw)


Fitting 3 folds for each of 27 candidates, totalling 81 fits

🔥 Best Parameters (RAW):
{'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 50}

📊 Evaluation Metrics for XGBRegressor (RAW):
MAE: 38.637932181248246
MSE: 2559.784599801195
RMSE: 50.59431390780188
R² Score: 0.997704351010824


c:\Users\bnsah\OneDrive\文档\PROJECTS(FINAL)\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [17:39:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "max_features" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
